# Develops the Calculation Model for the Online Calculator

In [1]:
import sys
from math import nan, isnan, inf
import numbers
from importlib import reload
import inspect
from pprint import pprint

import pandas as pd
import numpy as np
# import matplotlib pyplot commands
from matplotlib.pyplot import *
from IPython.display import Image, Markdown
from qgrid import show_grid as sh
 
# Show Plots in the Notebook
%matplotlib inline
 
# 'style' the plot like fivethirtyeight.com website
style.use('bmh')

In [2]:
rcParams['figure.figsize']= (10, 6)   # set Chart Size
rcParams['font.size'] = 14            # set Font size in Chart

In [3]:
# Access the directory where some utility modules are located in the
# actual heat pump calculator.
#sys.path.insert(0, '../../heat-pump-calc/heatpump/')
sys.path.insert(0, '../../heat-pump-calc/')

In [4]:
import heatpump.library as lib
reload(lib)

<module 'heatpump.library' from '../../heat-pump-calc/heatpump/library.py'>

In [5]:
import heatpump.hp_model
reload(heatpump.hp_model)
import heatpump.home_heat_model
reload(heatpump.home_heat_model)

<module 'heatpump.home_heat_model' from '../../heat-pump-calc/heatpump/home_heat_model.py'>

In [6]:
sh(lib.df_city[['Name', 'ElecUtilities']])

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [7]:
lib.fuels()

[('Electricity', 1),
 ('Natural Gas', 2),
 ('Propane', 3),
 ('#1 Oil', 4),
 ('#2 Oil', 5),
 ('Birch Wood', 6),
 ('Spruce Wood', 7),
 ('Wood Pellets', 8),
 ('Coal', 9),
 ('District Heat, Steam', 10),
 ('District Heat, Hot Water', 11)]

In [8]:
lib.city_from_id(45)

Name                                                            Ambler
Latitude                                                        67.087
Longitude                                                     -157.859
ERHRegionID                                                          5
WAPRegionID                                                          5
FuelRefer                                                        False
FuelCityID                                                         NaN
Oil1Price                                                          NaN
Oil2Price                                                          NaN
PropanePrice                                                       NaN
BirchPrice                                                         NaN
SprucePrice                                                        NaN
CoalPrice                                                          NaN
SteamPrice                                                         NaN
HotWat

In [9]:
lib.heat_pump_from_id(575)

brand                 Fujitsu
ahri_num              8703350
zones                  Single
outdoor_model       AOU12RLS3
indoor_model       ASU12RLS3Y
hspf                       14
in_pwr_5F_max             2.1
capacity_5F_max         16500
in_pwr_47F_min            0.2
Name: 575, dtype: object

In [10]:
util_id = 1
utility = lib.util_from_id(util_id)
utility

ID                                                              1
Name                                Chugach Electric- Residential
Active                                                       True
Type                                                            1
IsCommercial                                                False
ChargesRCC                                                   True
PCE                                                             0
CO2                                                           1.1
CustomerChg                                                     8
DemandCharge                                                  NaN
NameShort                                                  Chugac
Blocks          [(nan, 0.18647000112105194), (nan, nan), (nan,...
Name: 1, dtype: object

In [11]:
inputs1 = dict(
    city_id=1,
    utility_id=1,
    co2_lbs_per_kwh=1.1,
    exist_heat_fuel_id=2,
    exist_unit_fuel_cost=0.97852,
    exist_fuel_use=1600,
    exist_heat_effic=.8,
    exist_kwh_per_mmbtu=8,
    exist_is_point_source=False,
    includes_dhw=True,
    dhw_occupant_count=3,
    includes_dryer=True,
    elec_use_jan=550,
    elec_use_may=400,
    hp_model_id=575,
    low_temp_cutoff=5,
    garage_stall_count=2,
    garage_heated_by_hp=False,
    bldg_floor_area=3600,
    indoor_heat_setpoint=70,
    insul_level=3,  
    pct_exposed_to_hp=0.46,
    doors_open_to_adjacent=False,
    low_temp_acceptance=2,
    capital_cost=4500,
    rebate_dol=500,
    pct_financed=0.5,
    loan_term=10,
    loan_interest=0.05,
    hp_life=14,
    op_cost_chg=10,
    sales_tax=0.02,
    discount_rate=0.05,
    inflation_rate=0.02,
    fuel_esc_rate=0.03,
    elec_esc_rate=0.02,
)

In [12]:
inputs2 = dict(
    city_id=45,
    utility_id=202,
    co2_lbs_per_kwh=1.6,
    exist_heat_fuel_id=4,
    exist_unit_fuel_cost=8.0,
    exist_fuel_use=450,
    exist_heat_effic=.86,
    exist_kwh_per_mmbtu=8,
    exist_is_point_source=True,
    includes_dhw=False,
    dhw_occupant_count=3,
    includes_dryer=False,
    elec_use_jan=550,
    elec_use_may=400,
    hp_model_id=575,
    low_temp_cutoff=5,
    garage_stall_count=0,
    garage_heated_by_hp=False,
    bldg_floor_area=800,
    indoor_heat_setpoint=70,
    insul_level=2,  
    pct_exposed_to_hp=1.0,
    doors_open_to_adjacent=False,
    low_temp_acceptance=3,
    capital_cost=6500,
    rebate_dol=0,
    pct_financed=0.0,
    loan_term=10,
    loan_interest=0.05,
    hp_life=14,
    op_cost_chg=0,
    sales_tax=0.00,
    discount_rate=0.05,
    inflation_rate=0.02,
    fuel_esc_rate=0.03,
    elec_esc_rate=0.02,
)

In [13]:
mc = heatpump.hp_model.HP_model(**inputs2)
mc.run()
pprint(mc.summary)
mc.df_mo_dol_base

{'co2_driving_miles_saved': 987.1566051726571,
 'co2_lbs_saved': 880.4143742987326,
 'cop': 2.836600461342373,
 'design_heat_load': 21478.25351576897,
 'design_heat_temp': -49.900000000000006,
 'elec_rate_avg_base': 0.23200883941833067,
 'elec_rate_avg_hp': 0.28991140424534884,
 'elec_use_base': 5261.587156420448,
 'elec_use_chg': 2394.8579858264475,
 'elec_use_hp': 7656.445142246896,
 'fuel_desc': '#1 Oil',
 'fuel_unit': 'gallons',
 'fuel_use_base': 449.99999999999994,
 'fuel_use_chg': -212.53824980065627,
 'fuel_use_hp': 237.46175019934367,
 'hp_max_capacity_5F': 15046.92,
 'irr': 0.09915273879411424,
 'irr_no_pce': 0.05443037924944405,
 'max_hp_reached': False,
 'npv': 2459.449974522957,
 'npv_no_pce': 207.62689740143685}


,elec_kwh,elec_kw,elec_dol,elec_dol_no_pce,secondary_fuel_units,secondary_fuel_dol,total_dol,total_dol_no_pce
Jan,550.000000,1.883562,138.930000,287.480004,78.222091,625.776727,764.706727,913.256731
Feb,484.208794,1.658249,109.831203,253.689640,56.307711,450.461685,560.292888,704.151325
Mar,469.285322,1.607142,106.600272,246.024945,47.888818,383.110542,489.710814,629.135487
Apr,430.885219,1.475634,98.286650,226.302652,40.416905,323.335239,421.621889,549.637891
May,400.000000,1.369863,91.600000,210.440003,21.523920,172.191357,263.791357,382.631360
Jun,352.407154,1.206874,81.296148,185.996316,7.619120,60.952959,142.249107,246.949275
Jul,358.463021,1.227613,82.607244,189.106610,8.924464,71.395713,154.002956,260.502323
Aug,370.683244,1.269463,85.252922,195.382917,13.548163,108.385305,193.638227,303.768221
Sep,396.040992,1.356305,90.742874,208.406656,25.300050,202.400397,293.143271,410.807053
Oct,437.025287,1.496662,99.615974,229.456190,32.861796,262.894368,362.510342,492.350558


In [14]:
mc.df_mo_dol_hp

,elec_kwh,elec_kw,elec_dol,elec_dol_no_pce,secondary_fuel_units,secondary_fuel_dol,total_dol,total_dol_no_pce
Jan,635.560945,3.045697,182.874102,331.424105,70.822827,566.582614,749.456716,898.006720
Feb,668.064393,2.826424,199.567873,348.117877,41.181945,329.455557,529.023430,677.573434
Mar,816.149266,2.874737,264.009337,412.559341,19.646084,157.168669,421.178006,569.728010
Apr,725.603633,2.749623,226.559663,375.109667,14.576854,116.614832,343.174495,491.724499
May,606.314911,2.574699,167.853339,316.403342,1.695108,13.560862,181.414201,329.964205
Jun,426.070938,1.691062,97.244358,223.830037,0.000000,0.000000,97.244358,223.830037
Jul,444.459354,1.719351,101.225450,233.274327,0.000000,0.000000,101.225450,233.274327
Aug,505.632318,2.123970,116.142758,264.692762,0.000000,0.000000,116.142758,264.692762
Sep,661.324561,2.301645,196.106295,344.656299,0.000000,0.000000,196.106295,344.656299
Oct,811.166379,2.738900,261.948415,410.498419,0.000000,0.000000,261.948415,410.498419


In [15]:
mc.df_cash_flow

,initial_cost,loan_cost,operating_cost,fuel_cost,elec_cost,elec_cost_no_pce,cash_flow,cash_flow_no_pce
year,,,,,,,,
0,-6500.0,0.0,0.0,0.000000,-0.000000,-0.000000,-6500.000000,-6500.000000
1,0.0,-0.0,0.0,1700.305998,-998.956033,-1201.474158,701.349965,498.831840
2,0.0,-0.0,0.0,1751.315178,-1018.935154,-1225.503641,732.380025,525.811537
3,0.0,-0.0,0.0,1803.854634,-1039.313857,-1250.013714,764.540777,553.840920
4,0.0,-0.0,0.0,1857.970273,-1060.100134,-1275.013988,797.870139,582.956285
5,0.0,-0.0,0.0,1913.709381,-1081.302137,-1300.514268,832.407244,613.195113
6,0.0,-0.0,0.0,1971.120662,-1102.928179,-1326.524553,868.192483,644.596109
7,0.0,-0.0,0.0,2030.254282,-1124.986743,-1353.055044,905.267539,677.199238
8,0.0,-0.0,0.0,2091.161911,-1147.486478,-1380.116145,943.675433,711.045765


In [16]:
mc.df_mo_en_base

,hp_load_mmbtu,secondary_load_mmbtu,hp_kwh,secondary_fuel_mmbtu,secondary_kwh,hp_kw,secondary_fuel_units,cop,total_kwh,co2_lbs
Jan,0.0,9.246533,0.0,10.751783,73.972266,0.0,78.222091,NaN,73.972266,1852.618197
Feb,0.0,6.656062,0.0,7.739607,53.248499,0.0,56.307711,NaN,53.248499,1333.596279
Mar,0.0,5.660876,0.0,6.582414,45.287007,0.0,47.888818,NaN,45.287007,1134.202554
Apr,0.0,4.777631,0.0,5.555384,38.221045,0.0,40.416905,NaN,38.221045,957.237177
May,0.0,2.544315,0.0,2.958506,20.354520,0.0,21.523920,NaN,20.354520,509.774217
Jun,0.0,0.900646,0.0,1.047263,7.205171,0.0,7.619120,NaN,7.205171,180.451838
Jul,0.0,1.054949,0.0,1.226685,8.439596,0.0,8.924464,NaN,8.439596,211.367714
Aug,0.0,1.601511,0.0,1.862222,12.812088,0.0,13.548163,NaN,12.812088,320.875768
Sep,0.0,2.990686,0.0,3.477542,23.925492,0.0,25.300050,NaN,23.925492,599.208378
Oct,0.0,3.884551,0.0,4.516920,31.076407,0.0,32.861796,NaN,31.076407,778.301379


In [17]:
mc.df_mo_en_hp

,hp_load_mmbtu,secondary_load_mmbtu,hp_kwh,secondary_fuel_mmbtu,secondary_kwh,hp_kw,secondary_fuel_units,cop,total_kwh,co2_lbs
Jan,0.874658,8.371876,92.558205,9.734739,66.975006,1.162136,70.822827,2.769581,159.533211,1825.466568
Feb,1.787997,4.868065,198.159578,5.660541,38.944520,1.168175,41.181945,2.644495,237.104098,1292.411766
Mar,3.338537,2.322338,373.572243,2.700393,18.578707,1.267596,19.646084,2.619224,392.150951,1063.014991
Apr,3.054519,1.723111,319.154568,2.003618,13.784890,1.273989,14.576854,2.805000,332.939458,855.886674
May,2.343938,0.200377,225.066419,0.232996,1.603012,1.204836,1.695108,3.052295,226.669431,400.253337
Jun,0.900646,0.000000,80.868956,0.000000,0.000000,0.484188,0.000000,3.264100,80.868956,129.390329
Jul,1.054949,0.000000,94.435928,0.000000,0.000000,0.491738,0.000000,3.274051,94.435928,151.097485
Aug,1.601511,0.000000,147.761162,0.000000,0.000000,0.854507,0.000000,3.176586,147.761162,236.417859
Sep,2.990686,0.000000,289.209061,0.000000,0.000000,0.945340,0.000000,3.030749,289.209061,462.734497
Oct,3.884551,0.000000,405.217499,0.000000,0.000000,1.242238,0.000000,2.809594,405.217499,648.347999


In [18]:
mc.city

Name                                                            Ambler
Latitude                                                        67.087
Longitude                                                     -157.859
ERHRegionID                                                          5
WAPRegionID                                                          5
FuelRefer                                                        False
FuelCityID                                                         NaN
Oil1Price                                                          NaN
Oil2Price                                                          NaN
PropanePrice                                                       NaN
BirchPrice                                                         NaN
SprucePrice                                                        NaN
CoalPrice                                                          NaN
SteamPrice                                                         NaN
HotWat

In [19]:
# Model an average Enstar Home, probably somewhere between an
# insulation level 1 and 2.  Space Heating for Average Enstar
# House is about 1326 CCF, from "accessible_UA.ipynb"
m = heatpump.hp_model.HP_model(**inputs1)
m.insul_level = 2
m.exist_heat_effic = 0.76
m.bldg_floor_area = 2100
m.garage_stall_count = 1
m.exist_fuel_use = None
m.run()
f_level2 = m.summary['fuel_use_base']

m.insul_level = 1
m.run()
f_level1 = m.summary['fuel_use_base']

# Assuming 2/3 Level 2 and 1/3 Level 1
print(0.67 * f_level2 + 0.33 * f_level1)

1248.221167662789


In [20]:
# Check whether COP is getting modeled correctly
m.df_hourly.plot(x='db_temp', y='cop', marker='.', linewidth=0)

AttributeError: 'HP_model' object has no attribute 'df_hourly'